In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import json
from catboost import CatBoostClassifier, CatBoostRegressor
import numpy as np


from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, mean_squared_error, mean_absolute_error

In [ ]:
def correct_parse_xml_data(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    
    sentences_data = []
    
    for sentence_idx, sentence in enumerate(root.findall('.//sentence')):
        elements = list(sentence)
        words_in_sentence = []
        
        # Собираем все слова и их позиции
        word_elements = [(i, elem) for i, elem in enumerate(elements) if elem.tag == 'word']
        
        for word_pos, word_elem in word_elements:
            original = word_elem.get('original', '')
            has_stress = word_elem.get('nucleus') == '2'
            
            # Лингвистические характеристики
            dictitem = word_elem.find('dictitem')
            if dictitem is not None:
                pos = dictitem.get('subpart_of_speech', '')
                form = dictitem.get('form', '')
                gender = dictitem.get('genesys', '')
                semantics1 = dictitem.get('semantics1', '')
                semantics2 = dictitem.get('semantics2', '')
            else:
                pos = form = gender = semantics1 = semantics2 = ''
            
            # Правильно определяем паузу после слова
            pause_after = -1
            # Ищем следующие элементы после текущего слова
            for next_pos in range(word_pos + 1, len(elements)):
                next_elem = elements[next_pos]
                if next_elem.tag == 'pause':
                    pause_time = next_elem.get('time')
                    if pause_time and pause_time.isdigit():
                        pause_after = int(pause_time)
                    break
                elif next_elem.tag == 'word':
                    # Следующее слово - значит паузы нет
                    break
            
            word_data = {
                'sentence_id': sentence_idx,
                'original': original,
                'position_in_sentence': len(words_in_sentence),
                'total_words_in_sentence': len(word_elements),
                'words_before': len(words_in_sentence),
                'words_after': len(word_elements) - len(words_in_sentence) - 1,
                'has_capital': original and original[0].isupper(),
                'word_length': len(original),
                'part_of_speech': pos,
                'form': form,
                'gender': gender,
                'semantics1': semantics1,
                'semantics2': semantics2,
                'phrasal_stress': has_stress,
                'pause_length': pause_after
            }
            
            words_in_sentence.append(word_data)
        
        sentences_data.extend(words_in_sentence)
    
    return pd.DataFrame(sentences_data)

# Загружаем исправленные данные
df_corrected = correct_parse_xml_data('gogol_utf8_cut.Result.xml')
print(f"Загружено {len(df_corrected)} слов")
print(f"Уникальных предложений: {df_corrected['sentence_id'].nunique()}")

# Статистика
print(f"\nФразовые ударения:")
print(df_corrected['phrasal_stress'].value_counts(normalize=True))

pause_data = df_corrected[df_corrected['pause_length'] > 0]
print(f"\nПаузы (только > 0):")
print(f"Всего пауз: {len(pause_data)}")
if len(pause_data) > 0:
    print(f"Средняя длина: {pause_data['pause_length'].mean():.1f} мс")
    print(f"Мин-Макс: {pause_data['pause_length'].min()}-{pause_data['pause_length'].max()} мс")
    print(f"Примеры пауз: {pause_data['pause_length'].value_counts().head()}")

In [ ]:
# Анализ данных
print("Распределение частей речи:")
print(df_corrected['part_of_speech'].value_counts().head(10))

print("\nРаспределение форм слов:")
print(df_corrected['form'].value_counts().head(10))

print("\nРаспределение родов:")
print(df_corrected['gender'].value_counts())

# Посмотрим на примеры с паузами
if len(pause_data) > 0:
    print("\nПримеры слов с паузами:")
    sample_pauses = pause_data[['original', 'pause_length', 'position_in_sentence', 'total_words_in_sentence']].head(10)
    print(sample_pauses)

In [ ]:
# Предобработка данных
print("Предобработка данных...")

# Заполняем пропуски
df = df_corrected.copy()
df.fillna('', inplace=True)

# Кодируем категориальные переменные
categorical_columns = ['part_of_speech', 'form', 'gender', 'semantics1', 'semantics2']

label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df[col + '_encoded'] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le

# Признаки для моделей
feature_columns = [
    'position_in_sentence', 
    'total_words_in_sentence',
    'words_before', 
    'words_after', 
    'has_capital', 
    'word_length'
] + [col + '_encoded' for col in categorical_columns]

print(f"Используется {len(feature_columns)} признаков:")
print(feature_columns)

# Подготовка данных для двух задач
print("\nПодготовка данных для двух задач...")

# Задача 1: Классификация фразового ударения
X_stress = df[feature_columns]
y_stress = df['phrasal_stress']

# Задача 2: Регрессия для длины пауз (только слова с паузами)
pause_data = df[df['pause_length'] > 0]
X_pause = pause_data[feature_columns]
y_pause = pause_data['pause_length']

print(f"Задача 1 (ударения): {X_stress.shape[0]} примеров")
print(f"Задача 2 (паузы): {X_pause.shape[0]} примеров")

# Разделение на train/test
X_train_stress, X_test_stress, y_train_stress, y_test_stress = train_test_split(
    X_stress, y_stress, test_size=0.2, random_state=42, stratify=y_stress
)

X_train_pause, X_test_pause, y_train_pause, y_test_pause = train_test_split(
    X_pause, y_pause, test_size=0.2, random_state=42
)

print(f"\nРазделение данных:")
print(f"Ударения - train: {X_train_stress.shape[0]}, test: {X_test_stress.shape[0]}")
print(f"Паузы - train: {X_train_pause.shape[0]}, test: {X_test_pause.shape[0]}")

# Обучение моделей
print("\nОбучение моделей...")

# Модель для фразового ударения
stress_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    random_state=42,
    verbose=100,
    class_weights=[1, 3]  # Учитываем несбалансированность классов
)

print("Обучение модели для фразового ударения...")
stress_model.fit(
    X_train_stress, y_train_stress,
    eval_set=(X_test_stress, y_test_stress),
    early_stopping_rounds=50
)

# Модель для длины пауз
pause_model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    random_state=42,
    verbose=100
)

print("\nОбучение модели для длины пауз...")
pause_model.fit(
    X_train_pause, y_train_pause,
    eval_set=(X_test_pause, y_test_pause),
    early_stopping_rounds=50
)

# Оценка моделей
print("\nОценка моделей...")

# Оценка классификации
y_pred_stress = stress_model.predict(X_test_stress)
print("Фразовое ударение - отчет классификации:")
print(classification_report(y_test_stress, y_pred_stress))

# Оценка регрессии
y_pred_pause = pause_model.predict(X_test_pause)
mse = mean_squared_error(y_test_pause, y_pred_pause)
mae = mean_absolute_error(y_test_pause, y_pred_pause)

print(f"\nДлина пауз - метрики регрессии:")
print(f"MSE: {mse:.2f}")
print(f"MAE: {mae:.2f} мс")
print(f"RMSE: {np.sqrt(mse):.2f} мс")

# Важность признаков
print("\nВажность признаков для фразового ударения:")
feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'importance': stress_model.get_feature_importance()
}).sort_values('importance', ascending=False)
print(feature_importance.head(10))

print("\nВажность признаков для длины пауз:")
feature_importance_pause = pd.DataFrame({
    'feature': feature_columns,
    'importance': pause_model.get_feature_importance()
}).sort_values('importance', ascending=False)
print(feature_importance_pause.head(10))

In [ ]:
# Функция для предсказания на новых данных и генерации JSON
def predict_and_create_json(text_data, stress_model, pause_model, label_encoders):
    """
    Предсказывает фразовые ударения и длины пауз для нового текста
    и возвращает результат в требуемом JSON-формате
    """
    # Здесь должна быть логика преобразования текста в признаки
    # Для демонстрации используем существующие данные
    
    # Берем случайное предложение из тестовой выборки для демонстрации
    sample_sentence = df[df['sentence_id'] == 0]  # Первое предложение
    
    result = []
    sentence_data = {"words": []}
    
    for _, word_row in sample_sentence.iterrows():
        # Подготавливаем признаки для предсказания
        word_features = pd.DataFrame([word_row[feature_columns]])
        
        # Предсказываем ударение
        stress_pred = stress_model.predict(word_features)[0]
        
        # Предсказываем длину паузы (только если модель обучена)
        pause_pred = -1  # по умолчанию -1 (нет паузы)
        if pause_model:
            pause_pred = pause_model.predict(word_features)[0]
            # Округляем до целых и убеждаемся, что не отрицательное
            pause_pred = max(0, int(round(pause_pred)))
        
        # Добавляем слово в результат
        sentence_data["words"].append({
            "content": word_row['original'],
            "phrasal_stress": bool(stress_pred),
            "pause_len": pause_pred
        })
    
    result.append(sentence_data)
    
    return result

# Генерируем пример JSON-результата
print("\nГенерация примера JSON-результата...")
json_example = predict_and_create_json(None, stress_model, pause_model, label_encoders)

print("Пример JSON-структуры:")
print(json.dumps(json_example, indent=4, ensure_ascii=False))

# Сохраняем модели для будущего использования
print("\nСохранение моделей...")
stress_model.save_model('stress_model.cbm')
pause_model.save_model('pause_model.cbm')

print("Модели сохранены как 'stress_model.cbm' и 'pause_model.cbm'")

# Сохраняем кодировщики
import joblib
joblib.dump(label_encoders, 'label_encoders.pkl')
print("Кодировщики сохранены как 'label_encoders.pkl'")

In [ ]:
# Сохраняем JSON в файл
def save_json_result(json_data, filename='result.json'):
    """
    Сохраняет JSON-результат в файл с правильным форматированием
    """
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(json_data, f, indent=4, ensure_ascii=False)
    
    print(f"JSON-результат сохранен в файл: {filename}")

# Сохраняем наш пример
save_json_result(json_example, 'example_result.json')

# Теперь создадим более реалистичный пример - обработаем несколько предложений
def process_multiple_sentences(sentence_ids, stress_model, pause_model, label_encoders, df):
    """
    Обрабатывает несколько предложений и возвращает JSON-результат
    """
    result = []
    
    for sentence_id in sentence_ids:
        sentence_data = df[df['sentence_id'] == sentence_id]
        
        if len(sentence_data) == 0:
            continue
            
        sentence_result = {"words": []}
        
        for _, word_row in sentence_data.iterrows():
            # Подготавливаем признаки для предсказания
            word_features = pd.DataFrame([word_row[feature_columns]])
            
            # Предсказываем ударение
            stress_pred = stress_model.predict(word_features)[0]
            
            # Предсказываем длину паузы
            pause_pred = -1
            if pause_model:
                predicted_pause = pause_model.predict(word_features)[0]
                # Округляем и проверяем, что не отрицательное
                pause_pred = max(0, int(round(predicted_pause)))
            
            # Добавляем слово в результат
            sentence_result["words"].append({
                "content": word_row['original'],
                "phrasal_stress": bool(stress_pred),
                "pause_len": pause_pred
            })
        
        result.append(sentence_result)
    
    return result

# Обрабатываем несколько предложений для демонстрации
print("\nОбработка нескольких предложений для демонстрации...")
sample_sentences = [0, 1, 2, 3]  # Первые 4 предложения
multiple_results = process_multiple_sentences(sample_sentences, stress_model, pause_model, label_encoders, df)

# Сохраняем результат с несколькими предложениями
save_json_result(multiple_results, 'multiple_sentences_result.json')

print("Содержимое файла multiple_sentences_result.json:")
print(json.dumps(multiple_results, indent=4, ensure_ascii=False)[:1000] + "...")  # Показываем начало

# Также создадим функцию для обработки всего датасета (будет долго, но для полноты)
def create_final_submission_json(stress_model, pause_model, label_encoders, df, output_file='final_submission.json'):
    """
    Создает финальный JSON для submission на основе всех данных
    В реальной ситуации здесь нужно обрабатывать тестовые данные
    """
    print(f"Создание финального submission файла...")
    
    # Для демонстрации возьмем только первые 100 предложений
    # В реальном задании нужно обработать всю тестовую выборку
    demo_sentence_ids = df['sentence_id'].unique()[:100]
    
    result = process_multiple_sentences(demo_sentence_ids, stress_model, pause_model, label_encoders, df)
    
    # Сохраняем
    save_json_result(result, output_file)
    
    print(f"Файл {output_file} создан с {len(result)} предложениями")
    return result

# Создаем демо-версию для submission
final_result = create_final_submission_json(stress_model, pause_model, label_encoders, df, 'lab_submission.json')

# Проверяем, что файлы создались
import os
print(f"\nСозданные файлы:")
for file in ['example_result.json', 'multiple_sentences_result.json', 'lab_submission.json']:
    if os.path.exists(file):
        file_size = os.path.getsize(file)
        print(f"✓ {file} ({file_size} bytes)")
    else:
        print(f"✗ {file} - не найден")